In [1]:
# Run it once (in one session)
!pip install decord
!pip install einops
!pip install icecream
!pip install rarfile
!pip install unrar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 76.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# Imports
import torch
from torch import nn, einsum
from torch.nn import functional as F
from torch.utils.tensorboard import SummaryWriter
import torchvision as tv
from torch.utils.data import random_split, DataLoader,Dataset
import time
import random
import math
import decord
import numpy as np
import gc
from einops import rearrange, repeat,reduce
from einops.layers.torch import Rearrange
from PIL import Image
from tqdm.notebook import tqdm
from icecream import ic
from torchvision.datasets import DatasetFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, random_split
import tensorflow as tf
import os
import rarfile
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, random_split
import torch.utils.data as data
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import shutil

import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
# set device
device ='cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cpu'

# DOWNLOADING DATA FROM SOURCE WEBSITE

In [2]:
# once per session/runtime
!wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar 

--2023-05-30 20:35:53--  http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
Resolving serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)... 128.148.254.114
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar [following]
--2023-05-30 20:35:53--  https://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2124008126 (2.0G)
Saving to: ‘hmdb51_org.rar’

hmdb51_org.rar      100%[===================>]   1.98G  84.2MB/s    in 21s     

2023-05-30 20:36:14 (96.5 MB/s) - ‘hmdb51_org.rar’ saved [2124008126/2124008126]



# DATA DIRECTORY SETTINGS

In [ ]:
# once per session/runtime

rar_path = '/content/hmdb51_org.rar'  
extract_path = '/content/dataset' 

with rarfile.RarFile(rar_path, 'r') as rar:
    rar.extractall(extract_path)

In [ ]:
# once per session/runtime

direcs = os.listdir(extract_path)

for i in direcs:
  with rarfile.RarFile(f"dataset/{i}", 'r') as rar:
    rar.extractall(f"data/{i.split('.')[0]}")

In [ ]:
# once per session/runtime

!mkdir data_set

In [ ]:
# once per session/runtime

# Define the path to the dataset
data_path = '/content/data'
data_set = '/content/data_set'

In [ ]:
data_path = '/content/data'  # Path to the original dataset
dataset_dir = '/content/data_set'  # Destination directory for the modified dataset

# Iterate over the class folders in the original dataset
for class_folder in os.listdir(data_path):
    class_folder_path = os.path.join(data_path, class_folder)

    # Iterate over the subfolders within each class folder
    for subfolder in os.listdir(class_folder_path):
        subfolder_path = os.path.join(class_folder_path, subfolder)

        # Iterate over the files within each subfolder
        for file_name in os.listdir(subfolder_path):
            file_path = os.path.join(subfolder_path, file_name)

            # Create the destination directory in the modified dataset
            dest_dir = os.path.join(dataset_dir, subfolder)
            os.makedirs(dest_dir, exist_ok=True)

            # Move the file to the destination directory
            shutil.move(file_path, dest_dir)

print("Dataset modification complete.") # success message

Dataset modification complete.


# DATA LOADING

In [ ]:
# Dataset Class
class HMDB51Dataset(data.Dataset):
    def __init__(self, dataset_dir, frames_per_clip=16):
        super().__init__()
        self.dataset_dir = dataset_dir
        self.frames_per_clip = frames_per_clip
        self.video_list = []
        self.labels = []

        # Get the list of video directories
        video_dirs = sorted(os.listdir(dataset_dir))
        
        for label, video_dir in enumerate(video_dirs):
            video_files = os.listdir(os.path.join(dataset_dir, video_dir))
            self.video_list.extend([os.path.join(video_dir, video_file) for video_file in video_files])
            self.labels.extend([label] * len(video_files))

        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    def __len__(self):
        return len(self.video_list)

    def __getitem__(self, idx):
        video_path = os.path.join(self.dataset_dir, self.video_list[idx])
        vid = decord.VideoReader(video_path, ctx=decord.cpu(0))
        nframes = len(vid)

        # If the number of frames in the video is less than frames_per_clip, repeat the frames
        if nframes <= self.frames_per_clip:
            frame_idxs = torch.arange(0, self.frames_per_clip) % nframes
        # Else, sample uniformly separated frames
        else:
            frame_idxs = torch.linspace(0, nframes - 1, self.frames_per_clip).long()

        frames = []
        for frame_idx in frame_idxs:
            frame_idx = frame_idx.item()  # Convert to scalar value
            frame = Image.fromarray(vid[frame_idx].asnumpy())
            frame = self.transform(frame)
            frames.append(frame)

        frames = torch.stack(frames)

        label = self.labels[idx]
        return frames, label

In [ ]:
# Directory of the HMDB51 dataset
dataset_dir = "/content/data_set"

# Instantiate the dataset
hmdb51_dataset = HMDB51Dataset(dataset_dir)

# TRAIN TEST SPLIT

In [ ]:
# Split the dataset into train, validation, and test sets
train_len = int(0.85 * len(hmdb51_dataset))
train_data, val_data = torch.utils.data.random_split(hmdb51_dataset, [train_len, len(hmdb51_dataset) - train_len])

# Data loading parameters
batch_size = 64
test_batch_size = 1
num_workers = 0
pin_memory = True
num_classes = len(set(hmdb51_dataset.labels))

frames_per_clip = 16

In [ ]:
# Dataloaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory)
test_loader = torch.utils.data.DataLoader(hmdb51_dataset, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)

In [ ]:
# Instantiate and create train-val-test split
train_val_data = HMDB51Dataset(dataset_dir, frames_per_clip=frames_per_clip)
train_len = int(0.85 * len(train_val_data))
train_val_split = [train_len, len(train_val_data) - train_len]
train_data, val_data = random_split(train_val_data, train_val_split)
test_data = HMDB51Dataset(dataset_dir, frames_per_clip=frames_per_clip)

# Print the number of samples in each split
print(f"Train samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")
print(f"Test samples: {len(test_data)}")

Train samples: 5751
Validation samples: 1015
Test samples: 6766


# DEFINING MODEL

In [ ]:
class MLP(nn.Module):
    """
    Builds a simple feed forward network
    Args:
    - dim: (int) - inner dimension of embeddings
    - inner_dim: (int) - dimension of transformer head
    - n_class: (int) - number of output classes
    - encoder: the DinoVisionTransformer encoder
    """
    def __init__(self, dim, inner_dim, n_class, encoder):
        super().__init__()
        self.encoder = encoder
        self.mlp = nn.Sequential(
            nn.Linear(dim, n_class)
        )

    def forward(self, x):
        B, T, C, H, W = x.shape
        x = x.reshape(B*T, C, H, W)
        output = self.encoder(x)
        output = output.reshape(B, T, -1)
        avg = output.mean(dim=1)  # Average pooling over time
        return self.mlp(avg)

In [ ]:
# Instantiate the model
dinov2_vits14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14', force_reload=True)
dinov2_vits14.to(device)
for param in dinov2_vits14.parameters():
    param.requires_grad = False

model = MLP(384, 512, 101, dinov2_vits14)
model.to(device)

Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip


MLP(
  (encoder): DinoVisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
      (norm): Identity()
    )
    (blocks): ModuleList(
      (0-11): 12 x NestedTensorBlock(
        (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (attn): MemEffAttention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
        (

## SPECIFYING HYPER-PARAMETERS

In [ ]:
# Define the loss function and optimizer
loss_criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.01)

# MODEL TRAINING & EVALUATION

In [ ]:
# Training loop
epochs = 5
for epoch in tqdm(range(1, epochs + 1)):
    model.train()
    total_epoch_loss = 0
    for batch_id, (video_data, labels) in enumerate(train_loader):
        video_data, labels = video_data.to(device), labels.to(device)
        optimizer.zero_grad()
        prediction = model(video_data)
        loss = loss_criterion(prediction, labels)
        loss.backward()
        optimizer.step()
        total_epoch_loss += loss.item()
        # Add any additional training metrics/logging you need
        
    # Perform validation at the end of each epoch
    model.eval()
    total_loss = 0
    corrects = 0
    with torch.no_grad():
        for batch_id, (video_data, labels) in enumerate(val_loader):
            video_data, labels = video_data.to(device), labels.to(device)
            prediction = model(video_data)
            loss = loss_criterion(prediction, labels)
            total_loss += loss.item()
            corrects += (torch.argmax(prediction, dim=1) == labels).sum()

    accuracy = corrects / (len(val_loader) * batch_size)
    print(f"\n[Val Epoch]: {epoch} , Accuracy: {accuracy}, Valid Loss: {total_loss / len(val_loader)}")

In [ ]:
# Save the trained model
torch.save(model, "model.pth")